## w2v50のhtkdic表記文字列をhtkdicに追加する

In [1]:
cd ../data/external/

/Users/showta/project/wovle/data/external


### Mecabの準備

In [3]:
import MeCab

neologd_path = !echo `mecab-config --dicdir`"/mecab-ipadic-neologd"
print(neologd_path)
mecab = MeCab.Tagger('-d '+neologd_path[0])

['/usr/local/lib/mecab/dic/mecab-ipadic-neologd']


In [4]:
text = "きんいろモザイク"
text2 = mecab.parse(text)
text2

'きんいろモザイク\t名詞,固有名詞,一般,*,*,*,きんいろモザイク,キンイロモザイク,キンイロモザイク\nEOS\n'

In [5]:
text2.split(',')[7:9]

['キンイロモザイク', 'キンイロモザイク\nEOS\n']

In [6]:
mecab.parse("獨協大")

'獨協大\t名詞,固有名詞,組織,*,*,*,獨協大,ドッキョウダイ,ドッキョーダイ\nEOS\n'

In [7]:
mecab.parse("獨協大").split(",")[8].split('\n')[0]

'ドッキョーダイ'

In [8]:
def convertToKatakana(string):
    return mecab.parse(string).split(",")[8].split('\n')[0]

convertToKatakana("東海道膝栗毛")

'トーカイドー'

In [9]:
mecab.parse("東海道膝栗毛")

'東海道\t名詞,固有名詞,地域,一般,*,*,東海道,トウカイドウ,トーカイドー\n膝栗毛\t名詞,一般,*,*,*,*,膝栗毛,ヒザクリゲ,ヒザクリゲ\nEOS\n'

In [10]:
def convertToKatakana(string):
    texts = mecab.parse(string).split('\n')[:-2]
    return ''.join(map(lambda x:x.split(",")[8], texts))
convertToKatakana("東海道膝栗毛")

'トーカイドーヒザクリゲ'

In [11]:
convertToKatakana("今日の天気は晴れのち雨かもね")

'キョーノテンキワハレノチウカモネ'

In [12]:
convertToKatakana("霧雨")

'キリサメ'

In [13]:
convertToKatakana("雨ふり")

'アメフリ'

In [14]:
mecab.parse("45")

'45\t名詞,数,*,*,*,*,*\nEOS\n'

In [15]:
mecab.parse("45").split('\n')[:-2]

['45\t名詞,数,*,*,*,*,*']

In [16]:
mecab.parse("愛").split('\n')[:-2]

['愛\t名詞,一般,*,*,*,*,愛,アイ,アイ']

In [17]:
mecab.parse("四十五").split('\n')[:-2]

['四\t名詞,数,*,*,*,*,四,ヨン,ヨン',
 '十\t名詞,数,*,*,*,*,十,ジュウ,ジュー',
 '五\t名詞,数,*,*,*,*,五,ゴ,ゴ']

In [18]:
mecab_y = MeCab.Tagger('-Ochasen')
mecab_y.parse("45")

'45\t45\t45\t名詞-数\t\t\nEOS\n'

諦める　記号はすべてdelete

### カタカナ表記を追加

In [33]:
# https://github.com/naoh16/ouslp/blob/master/scripts/vocab2htkdic.pl
!wget https://raw.githubusercontent.com/naoh16/ouslp/master/scripts/vocab2htkdic.pl

--2018-02-24 21:08:49--  https://raw.githubusercontent.com/naoh16/ouslp/master/scripts/vocab2htkdic.pl
raw.githubusercontent.com (raw.githubusercontent.com) をDNSに問いあわせています... 151.101.228.133
raw.githubusercontent.com (raw.githubusercontent.com)|151.101.228.133|:443 に接続しています... 接続しました。
HTTP による接続要求を送信しました、応答を待っています... 200 OK
長さ: 4762 (4.7K) [text/plain]
`vocab2htkdic.pl' に保存中

vocab2htkdic.pl     100%[===================>]   4.65K  --.-KB/s 時間 0s         

2018-02-24 21:08:49 (28.6 MB/s) - `vocab2htkdic.pl' へ保存完了 [4762/4762]



use open IO => ':encoding(UTF-8)'; を追加した

d y -> d u, ヮ追加

+だとダブるので@+@で区切る

In [23]:
!echo "DuWa@+@デュヮ" > temp.txt

In [24]:
!perl vocab2htkdic.pl temp.txt

DuWa@+@デュヮ	[DuWa]	d u w a 


In [25]:
import jaconv

def judge(c):
    if 'ぁ' <= c <= 'ゟ' or 'ァ' <= c <= 'ヿ' or '･' <= c <= 'ﾟ':
        return True
    return False

def convNihongo(c):
    if 'ぁ' <= c <= 'ゟ':
        return jaconv.hira2kata(c)
    if 'ァ' <= c <= 'ヿ':
        return c
    if '･' <= c <= 'ﾟ':
        return jaconv.h2z(c)
def convertToKatakana(string):
    texts = mecab.parse(string).split('\n')[:-2]
    for text in texts:
        if len(text.split(",")) < 8:
            word = text.split('\t')[0].replace('\n','')
            for c in word:
                if not judge(c):
                    raise Exception()
            return convNihongo(word)
    return ''.join(map(lambda x: x.split('\t')[1].split(",")[8], texts))
convertToKatakana("東海道膝栗毛")

'トーカイドーヒザクリゲ'

In [26]:
w = pd.read_pickle('./w2v50.pd')

In [27]:
w.index

Index(['!', '!!', '!!!', '!!!!', '!!!!」', '!!!」', '!!!』', '!!!』（', '!!!（',
       '!!!）',
       ...
       '･･････。', 'ｰ', 'ｸﾗﾌﾞ', 'ｼﾞﾑ', 'ﾟ', '￡', '￢', '￥', '𢷋', '𣰰'],
      dtype='object', name='単語', length=335476)

In [29]:
cnt = 0
with open('vocab_with_kana.txt', 'w') as fh2:
    for word in monitor(w.index):
        try:
            fh2.write(word.replace("\n","")+"@+@"+convertToKatakana(word)+"\n")
        except Exception:
            cnt += 1
print("error ="+str(cnt))

100%|██████████| 335476/335476 [00:09<00:00, 36820.58it/s] 

error =28530


In [30]:
!wc -l ./vocab_with_kana.txt

  306946 ./vocab_with_kana.txt


In [31]:
!head ./vocab_with_kana.txt

$0@+@ゼロドル
$1@+@イチドル
$10@+@ジュウドル
$100@+@ヒャクドル
$1000@+@センドル
$12@+@ジュウニドル
$15@+@ジュウゴドル
$150@+@ヒャクゴジュウドル
$2@+@ニドル
$20@+@ニジュウドル


In [32]:
!perl vocab2htkdic.pl vocab_with_kana.txt > vocab.htkdic

¨@+@¨
´@+@´
‐@+@‐
―@+@―
――@+@――
‘@+@‘
’@+@’
“@+@“
”@+@”
‥@+@‥
…@+@…
′@+@′
※@+@※
←@+@←
↑@+@↑
→@+@→
↓@+@↓
─@+@─
■@+@■
□@+@□
▲@+@▲
△@+@△
▼@+@▼
▽@+@▽
◆@+@◆
◇@+@◇
○@+@○
◎@+@◎
●@+@●
★@+@★
☆@+@☆
　@+@　
。@+@。
〃@+@〃
々@+@々
〆@+@〆
〇@+@〇
〈@+@〈
〉@+@〉
《@+@《
》@+@》
「@+@「
」@+@」
『@+@『
』@+@』
【@+@【
】@+@】
〔@+@〔
〕@+@〕
〜@+@〜
っ@+@ッ
゛@+@゛
゜@+@゜
ゝ@+@ゝ
ゞ@+@ゞ
ッ@+@ッ
ヵ@+@ヵ
ヶ@+@ヶ
・@+@・
ー@+@ー
ーー@+@ーー
ヽ@+@ヽ
仝@+@仝
！@+@！
（@+@（
）@+@）
＊@+@＊
．@+@．
／@+@／
：@+@：
；@+@；
＜@+@＜
＝@+@＝
＞@+@＞
？@+@？
［@+@［
＼@+@＼
］@+@］
＾@+@＾
＿@+@＿
｛@+@｛
｜@+@｜
｝@+@｝
･@+@・
･･･@+@・・・
･･････@+@・・・・・・
ｰ@+@ー
ﾟ@+@ﾟ


（記号は面倒いので見捨てる。いままでありがとな…）

In [33]:
!head ./vocab.htkdic

$0@+@ゼロドル	[$0]	z e r o d o r u 
$1@+@イチドル	[$1]	i ch i d o r u 
$10@+@ジュウドル	[$10]	j u: d o r u 
$100@+@ヒャクドル	[$100]	hy a k u d o r u 
$1000@+@センドル	[$1000]	s e N d o r u 
$12@+@ジュウニドル	[$12]	j u: n i d o r u 
$15@+@ジュウゴドル	[$15]	j u: g o d o r u 
$150@+@ヒャクゴジュウドル	[$150]	hy a k u g o j u: d o r u 
$2@+@ニドル	[$2]	n i d o r u 
$20@+@ニジュウドル	[$20]	n i j u: d o r u 


In [34]:
with open('vocab.htkdic') as fin:
    with open('vocab2.htkdic','w') as fout:
        #fout.write("<s>	[]	sp_S\n</s>	[。]	sp_S\n、+補助記号	[、]	sp_S\n")
        for line in monitor(fin):
            ws = line.split('\t')[2].replace("\n","").split(" ")[:-1]
            if ws[0] == "":
                ws = ws[1:]
            if len(ws) == 1:
                ws[0] += "_S"
            else:
                ws[0] += "_B"
                ws[-1] += "_E"
                ws[1:-1] = map(lambda x:x+"_I", ws[1:-1])
            fout.write(
                line.split('\t')[0].split("@+@")[0]
                +"\t"
                +line.split('\t')[1]
                +"\t"
                +' '.join(ws)
                +"\n")

306858it [00:02, 148508.17it/s]


In [35]:
!head vocab2.htkdic

$0	[$0]	z_B e_I r_I o_I d_I o_I r_I u_E
$1	[$1]	i_B ch_I i_I d_I o_I r_I u_E
$10	[$10]	j_B u:_I d_I o_I r_I u_E
$100	[$100]	hy_B a_I k_I u_I d_I o_I r_I u_E
$1000	[$1000]	s_B e_I N_I d_I o_I r_I u_E
$12	[$12]	j_B u:_I n_I i_I d_I o_I r_I u_E
$15	[$15]	j_B u:_I g_I o_I d_I o_I r_I u_E
$150	[$150]	hy_B a_I k_I u_I g_I o_I j_I u:_I d_I o_I r_I u_E
$2	[$2]	n_B i_I d_I o_I r_I u_E
$20	[$20]	n_B i_I j_I u:_I d_I o_I r_I u_E


In [36]:
!head ./standard.htkdic

<s>	[]	sp_S
</s>	[。]	sp_S
、+補助記号	[、]	sp_S
×+名詞	[×]	b_B a_I ts_I u_E
Δ+記号	[Δ]	d_B e_I r_I u_I t_I a_E
Θ+記号	[Θ]	sh_B i:_I t_I a_E
Σ+記号	[Σ]	sh_B i_I g_I u_I m_I a_E
Φ+記号	[Φ]	f_B a_I i_E
Ψ+記号	[Ψ]	p_B u_I s_I a_I i_E
Ω+名詞	[Ω]	o:_B m_I u_E


In [37]:
standard = []
with open('./standard.htkdic') as fh:
    for line in monitor(fh):
        standard.append(line.split("+")[0])

64274it [00:00, 201086.27it/s]


In [38]:
w.index

Index(['!', '!!', '!!!', '!!!!', '!!!!」', '!!!」', '!!!』', '!!!』（', '!!!（',
       '!!!）',
       ...
       '･･････。', 'ｰ', 'ｸﾗﾌﾞ', 'ｼﾞﾑ', 'ﾟ', '￡', '￢', '￥', '𢷋', '𣰰'],
      dtype='object', name='単語', length=335476)

In [39]:
standard = standard[2:]

In [40]:
standard = set(standard)

In [92]:
cp ./standard.htkdic ./hybrid.htkdic

In [93]:
!head ./vocab2.htkdic

$0	[$0]	z_B e_I r_I o_I d_I o_I r_I u_E
$1	[$1]	i_B ch_I i_I d_I o_I r_I u_E
$10	[$10]	j_B u:_I d_I o_I r_I u_E
$100	[$100]	hy_B a_I k_I u_I d_I o_I r_I u_E
$1000	[$1000]	s_B e_I N_I d_I o_I r_I u_E
$12	[$12]	j_B u:_I n_I i_I d_I o_I r_I u_E
$15	[$15]	j_B u:_I g_I o_I d_I o_I r_I u_E
$150	[$150]	hy_B a_I k_I u_I g_I o_I j_I u:_I d_I o_I r_I u_E
$2	[$2]	n_B i_I d_I o_I r_I u_E
$20	[$20]	n_B i_I j_I u:_I d_I o_I r_I u_E


### スペースがあると動かなかったので修正

In [94]:
vocab = pd.read_csv('./vocab2.htkdic',sep='\t',header=None)

In [95]:
vocab.shape

(306858, 3)

In [96]:
vocab.head()

,0,1,2
0,$0,[$0],z_B e_I r_I o_I d_I o_I r_I u_E
1,$1,[$1],i_B ch_I i_I d_I o_I r_I u_E
2,$10,[$10],j_B u:_I d_I o_I r_I u_E
3,$100,[$100],hy_B a_I k_I u_I d_I o_I r_I u_E
4,$1000,[$1000],s_B e_I N_I d_I o_I r_I u_E


In [97]:
val = [ str(v[0]) for v in vocab[[0]].values]

In [98]:
rep = [v.replace(' ','') for v in monitor(val)]

100%|██████████| 306858/306858 [00:00<00:00, 1748239.59it/s]


In [99]:
vocab[[0]] = pd.DataFrame(rep)

In [100]:
val = [ str(v[0]) for v in vocab[[1]].values]

In [101]:
rep = [v.replace(' ','') for v in monitor(val)]

100%|██████████| 306858/306858 [00:00<00:00, 1816543.72it/s]


In [102]:
vocab[[1]] = pd.DataFrame(rep)

In [103]:
vocab.head()

,0,1,2
0,$0,[$0],z_B e_I r_I o_I d_I o_I r_I u_E
1,$1,[$1],i_B ch_I i_I d_I o_I r_I u_E
2,$10,[$10],j_B u:_I d_I o_I r_I u_E
3,$100,[$100],hy_B a_I k_I u_I d_I o_I r_I u_E
4,$1000,[$1000],s_B e_I N_I d_I o_I r_I u_E


In [104]:
vocab[vocab[0] == '.NETFramework']

,0,1,2
53,.NETFramework,[.NETFramework],d_B o_I q_I t_I o_I n_I e_I q_I t_I o_I f_I u_...


In [105]:
vocab.to_csv('./vocab2.htkdic',sep='\t',header=None,index=None)

In [106]:
!head ./vocab2.htkdic

$0	[$0]	z_B e_I r_I o_I d_I o_I r_I u_E
$1	[$1]	i_B ch_I i_I d_I o_I r_I u_E
$10	[$10]	j_B u:_I d_I o_I r_I u_E
$100	[$100]	hy_B a_I k_I u_I d_I o_I r_I u_E
$1000	[$1000]	s_B e_I N_I d_I o_I r_I u_E
$12	[$12]	j_B u:_I n_I i_I d_I o_I r_I u_E
$15	[$15]	j_B u:_I g_I o_I d_I o_I r_I u_E
$150	[$150]	hy_B a_I k_I u_I g_I o_I j_I u:_I d_I o_I r_I u_E
$2	[$2]	n_B i_I d_I o_I r_I u_E
$20	[$20]	n_B i_I j_I u:_I d_I o_I r_I u_E


#### 修正終わり

In [107]:
with open('./hybrid.htkdic','a') as fout:
    with open('./vocab2.htkdic') as fin:
        for line in fin:
            if line.split('\t')[0] not in standard:
                fout.write(line)

In [108]:
!head ./hybrid.htkdic

<s>	[]	sp_S
</s>	[。]	sp_S
、+補助記号	[、]	sp_S
×+名詞	[×]	b_B a_I ts_I u_E
Δ+記号	[Δ]	d_B e_I r_I u_I t_I a_E
Θ+記号	[Θ]	sh_B i:_I t_I a_E
Σ+記号	[Σ]	sh_B i_I g_I u_I m_I a_E
Φ+記号	[Φ]	f_B a_I i_E
Ψ+記号	[Ψ]	p_B u_I s_I a_I i_E
Ω+名詞	[Ω]	o:_B m_I u_E


In [109]:
!wc -l ./hybrid.htkdic

  322238 ./hybrid.htkdic


In [110]:
!wc -l ./standard.htkdic

   64274 ./standard.htkdic


In [111]:
!wc -l ./vocab2.htkdic

  306858 ./vocab2.htkdic


In [112]:
!head hybrid.htkdic

<s>	[]	sp_S
</s>	[。]	sp_S
、+補助記号	[、]	sp_S
×+名詞	[×]	b_B a_I ts_I u_E
Δ+記号	[Δ]	d_B e_I r_I u_I t_I a_E
Θ+記号	[Θ]	sh_B i:_I t_I a_E
Σ+記号	[Σ]	sh_B i_I g_I u_I m_I a_E
Φ+記号	[Φ]	f_B a_I i_E
Ψ+記号	[Ψ]	p_B u_I s_I a_I i_E
Ω+名詞	[Ω]	o:_B m_I u_E


In [113]:
!tail hybrid.htkdic

ｐ	[ｐ]	p_B i:_E
ｒ	[ｒ]	a:_B r_I u_E
ｓ	[ｓ]	e_B s_I u_E
ｔ	[ｔ]	t_B i:_E
ｖ	[ｖ]	b_B u_I i_E
ｗ	[ｗ]	d_B a_I b_I u_I ry_I u:_E
ｘ	[ｘ]	e_B q_I k_I u_I s_I u_E
ｸﾗﾌﾞ	[ｸﾗﾌﾞ]	k_B u_I r_I a_I b_I u_E
ｼﾞﾑ	[ｼﾞﾑ]	j_B i_I m_I u_E
￥	[￥]	e_B N_E


In [114]:
cat ./hybrid.htkdic | grep 'EmacsLisp'

EmacsLisp	[EmacsLisp]	i:_B m_I a_I q_I k_I u_I s_I u_I r_I i_I s_I u_I p_I u_E


In [115]:
cat ./hybrid.htkdic | grep 'du_'

In [116]:
cat ./hybrid.htkdic | grep 'Q&A'

Q&A	[Q&A]	Qa_B N_I d_I o_E


これだけおかしい
A-Zがどこで変換されてるのか忘れちゃったので手で直す…

In [125]:
cat ./hybrid.htkdic | sed s/"Qa_B N_I d_I o_E"/"ky_B a_I N_I d_I o_I e:_E"/g > buf

In [126]:
cat ./buf | grep 'Q&A'

Q&A	[Q&A]	ky_B a_I N_I d_I o_I e:_E


In [127]:
mv ./buf ./hybrid.htkdic